In [1343]:
import numpy as np
import math as m
import random
import pandas as pd
import random
import csv
from copy import deepcopy

### Read Data di Excel

In [1344]:
dataTrain = pd.read_excel('DatasetAI.xlsx', sheet_name=0)

In [1345]:
dataTrainList = dataTrain.values.tolist()

### Encode Binary
(1) Suhu, urutannya : Rendah, Normal, Tinggi. (2) Waktu, urutannya : Pagi, Siang, Sore, Malam. (3) Kondisi Langit, urutannya : Cerah, Berawan, Rintik, Hujan. (4) Kelembapan, urutannya : Rendah, Normal, Tinggi. (5)Terbang, urutannya: Ya, Tidak

In [1346]:
def encodeBinary(data):
    binary = []
    # SUHU
    if data[0] == 'Rendah':
        binary.extend([1,0,0])
    elif data[0] == 'Normal':
        binary.extend([0,1,0])
    else:
        binary.extend([0,0,1])
    
    # WAKTU
    if data[1] == 'Pagi':
        binary.extend([1,0,0,0])
    elif data[1] == 'Siang':
        binary.extend([0,1,0,0])
    elif data[1] == 'Sore':
        binary.extend([0,0,1,0])
    else:
        binary.extend([0,0,0,1])
    
    # KONDISI
    if data[2] == 'Cerah':
        binary.extend([1,0,0,0])
    elif data[2] == 'Berawan':
        binary.extend([0,1,0,0])
    elif data[2] == 'Rintik':
        binary.extend([0,0,1,0])
    else:
        binary.extend([0,0,0,1])

    # KELEMBAPAN
    if data[3] == 'Rendah':
        binary.extend([1,0,0])
    elif data[3] == 'Normal':
        binary.extend([0,1,0])
    else:
        binary.extend([0,0,1])
    
    # TERBANG(?)
    if data[4] == 'Ya':
        binary.append(1)
    else:
        binary.append(0)
    return binary

## Decode Binary

### Suhu

In [1347]:
def decSuhu(data):
    if data[0:3] == [1,0,0]:
        return ('Rendah')
    elif data[0:3] == [0,1,0]:
        return ('Normal')
    else:
        return ('Tinggi')

### Waktu

In [1348]:
def decWaktu(data):
    if data[3:7] == [1,0,0,0]:
        return ('Pagi')
    elif data[3:7] == [0,1,0,0]:
        return ('Siang')
    elif data[3:7] == [0,0,1,0]:
        return ('Sore')
    else:
        return ('Malam')

### Kondisi

In [1349]:
def decKondisi(data):
    if data[7:11] == [1,0,0,0]:
        return ('Cerah')
    elif data[7:11] == [0,1,0,0]:
        return ('Berawan')
    elif data[7:11] == [0,0,1,0]:
        return ('Rintik')
    else:
        return ('Hujan')

### Kelembapan

In [1350]:
def decKelembapan(data):
    if data[11:14] == [1,0,0]:
        return ('Rendah')
    elif data[11:14] == [0,1,0]:
        return ('Normal')
    else:
        return ('Tinggi')

### Klasifikasi

In [1351]:
def decKlasifikasi(data):
    if data[14] == 1:
        return ('Ya')
    else:
        return ('Tidak')

In [1352]:
def decodeBinary(data):
    dec = []
    for i in range(len(data)):
        decode = []
        decode.append(decSuhu(data[i]))
        decode.append(decWaktu(data[i]))
        decode.append(decKondisi(data[i]))
        decode.append(decKelembapan(data[i]))
        decode.append(decKlasifikasi(data[i]))
        dec.append(decode)
    return dec

### Convert Rule ke dalam Binary

In [1353]:
def convertRule(dataTrainList):
    dataBinary = []
    for i in range (len(dataTrainList)):
         dataBinary.append(encodeBinary(dataTrainList[i]))
    return dataBinary

### Buat Individu

In [1365]:
def buatKromosom(pop):
    gen = []
    data = [15,30,45,60]
    for i in range(pop):
        coba = []
        r = random.choice(data)
        for i in range(r):
            coba.append(random.randint(0,1))
        gen.append(coba)
    return gen

### Fitness

In [1355]:
def fitness(x, dataLatih):
    count = 0
    for i in range (len(dataLatih)):
        conn = False
        k = 0
        while(conn == False and len(x)!=k):
            temp1 = []
            temp2 = []
            for j in range(len(x[k])-1):
                if x[k][j] and dataLatih[i][j]:
                    temp1.append(1)
                else:
                    temp1.append(0)
    
            if temp1[0] or temp1[1] or temp1[2]:
                temp2.append(1)
            else:
                temp2.append(0)
            if temp1[3] or temp1[4] or temp1[5] or temp1[6]:
                temp2.append(1)
            else:
                temp2.append(0)
            if temp1[7] or temp1[8] or temp1[9] or temp1[10]:
                temp2.append(1)
            else:
                temp2.append(0)
            if temp1[11] or temp1[12] or temp1[13]:
                temp2.append(1)
            else:
                temp2.append(0)
            
            if (temp2[0] and temp2[1] and temp2[2] and temp2[3]) == 1:
                if x[k][-1] == dataLatih[i][-1]:
                    count += 1
                    conn = True
            elif len(x)==1 or (k==len(x)-1):
                if (temp2[0] and temp2[1] and temp2[2] and temp2[3]) == 0:
                    if x[k][-1] != dataLatih[i][-1]:
                        count += 1
            k += 1
    fitness = (count/len(dataLatih)) * 100
    
    if fitness != 0:
        dataCoba = deepcopy(dataTrainBinary)

        for k in range(len(dataCoba)):
            del dataCoba[k][14]

        listUji = cekData(dataCoba,x)

        itung = 0
        for i in range(len(dataCoba)):
            if listUji[i] == dataTrainBinary[i][14]:
                itung += 1

        fn = (itung/len(dataCoba))*100
        if (fn != fitness):
            fitness = fn
    
    return fitness

### Elitism

In [1356]:
def elitism(gen,fit):
    sort = sorted(zip(fit,gen), reverse=True)
    elitgen = []
    elitfit = []   
    for i in range(2):
        elitgen.append(sort[i][1])
        elitfit.append(sort[i][0])
    return elitgen, elitfit

### Tournament

In [1357]:
def tournament(data,pop,dataLatih):
    best = []
    N = len(data)
    for i in range(N):
        k = i-1
        bst = -1
        for j in range((N//2)-2):
            r = random.randint(0,N-1)
            if bst == -1 or fitness(pop[r],dataLatih)>fitness(pop[bst],dataLatih):
                bst = r
        if not best or best[k]!=data[bst]:
            best.append(data[bst])
        else:
            while(best[k]==data[bst]):
                r = random.randint(0,N-1)
                if bst != r:
                    bst = r
            best.append(data[bst])
    return best

### Crossover

In [1358]:
def crossover(x1,x2,pc):
    rr = random.uniform(0,1)
    if rr<=pc:
        R1 = random.randint(0,len(x1))
        R2 = random.randint(0,len(x1))
        while(R1==R2):
            R2 = random.randint(0,len(x1)-1)

        if R2<R1:
            R2, R1 = R1, R2

        A, B, C = deepcopy(x1[:R1]), deepcopy(x1[R1:R2]), deepcopy(x1[R2:])

        diff = R2-R1
        gap = diff % 15

        conn = False
        while conn != True:
            R11 = random.randint(0,len(x2))
            R12 = R11+gap
            conn = R12 < 15

        D, E, F = deepcopy(x2[:R11]), deepcopy(x2[R11:R12]), deepcopy(x2[R12:])

        aa = (A+E+C)
        ab = (D+B+F)
        
        return aa, ab
    else:
        return x1, x2

### Mutasi

In [1359]:
def mutasi(childm,lm):
    length = len(childm)
    for i in range(length):
        r = np.random.uniform(0,1)
        if r<=lm :
            if childm[i] == 1:
                childm[i] = 0
            else:
                childm[i] = 1
    return childm

### Pengecekan Data

In [1360]:
def cekData(dataUjiBinary,gensim):
    pdu = len(dataUjiBinary)
    listk = []
    for i in range(pdu):
        conn = False
        k = 0
        while(conn == False and len(gensim)!=k):
            temp1 = []
            temp2 = []
            for j in range(len(gensim[k])-1):
                if dataUjiBinary[i][j] and gensim[k][j]:
                    temp1.append(1)
                else:
                    temp1.append(0)
            if temp1[0] or temp1[1] or temp1[2]:
                temp2.append(1)
            else:
                temp2.append(0)
            if temp1[3] or temp1[4] or temp1[5] or temp1[6]:
                temp2.append(1)
            else:
                temp2.append(0)
            if temp1[7] or temp1[8] or temp1[9] or temp1[10]:
                temp2.append(1)
            else:
                temp2.append(0)
            if temp1[11] or temp1[12] or temp1[13]:
                temp2.append(1)
            else:
                temp2.append(0)

            if (temp2[0] and temp2[1] and temp2[2] and temp2[3]) == 1:
                if gensim[k][14] == 1:
                    listk.append(1)
                else:
                    listk.append(0)
                conn = True
            elif (len(gensim)==1 or k==len(gensim)-1):
                if (temp2[0] and temp2[1] and temp2[2] and temp2[3]) == 0:
                    if gensim[k][14] == 0:
                        listk.append(1)
                    else:
                        listk.append(0)
            k += 1
    return listk

# MAIN PROGRAM

In [1368]:
pop = 20
pmut = 1/pop
pcross = 0.7
generasi = 200
dataTrainBinary = convertRule(dataTrainList)

#Buat populasi untuk generasi pertama
gen = buatKromosom(pop)
    
# Perulangan Generasi
for g in range(generasi):
    
    #Cek Fitness/Akurasi populasi gen
    fit = []
    sa = []
    for i in range(pop):
        gensim = []
        pj = len(gen[i])//15
        lf = deepcopy(gen[i])
        for j in range(pj):
            sim = []
            for k in range(15):
                sim.append(lf[0])
                del lf[0]
            gensim.append(sim)
        sa.append(gensim)
        fit.append(fitness(gensim,dataTrainBinary))
    
    #Melakukan Elitism
    eg = []
    ef = []
    eg, ef = elitism(gen,fit)
    elit = sorted(zip(ef,eg), reverse=True)
#     print('Gen ',g)
#     print(*elit, sep = "\n")

    #SELEKSI PARENT
    psel = []
    psel = tournament(gen,sa,dataTrainBinary)

    #CROSSOVER
    child = []
    i = 0
    while (i!=len(psel)):
        child.extend(crossover(psel[i],psel[i+1],pcross))
        i = i+2

    #MUTASI
    for i in range(len(child)):
            r = np.random.uniform(0,1)
            if r<=pmut:
                child[i] = mutasi(child[i],pmut)

    #HITUNG FITNESS CHILD BARU
    fchild = []
    for i in range(pop):
        gensim = []
        pj = len(child[i])//15
        lf = deepcopy(child[i])
        for j in range(pj):
            sim = []
            for k in range(15):
                sim.append(lf[0])
                del lf[0]
            gensim.append(sim)
        fchild.append(fitness(gensim,dataTrainBinary))

    #Menentukan populasi untuk generasi selanjutnya
    yx = []
    fx = []
    yx = child + eg
    fx = fchild + ef
    bestgen = sorted(zip(fx,yx), reverse=True)[:pop]
    
    
    #Mendapat populasi baru untuk generasi selanjutnya
    gen = []
    for i in range(pop):
        gen.append(bestgen[i][1])

    if (g % 10 == 0):
        print('Best Generation -',g)
        print(*bestgen[0], sep = "\n")
        print()
        
print()
print('Best Fitness(Akurasi) dan Individu(Tree):')
print(bestgen[0][0],' - ',bestgen[0][1])

Best Generation - 0
63.74999999999999
[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1]

Best Generation - 10
70.0
[0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1]

Best Generation - 20
77.5
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1]

Best Generation - 30
80.0
[1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1

Best Generation - 130
83.75
[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,

### Test Menggunakan Data Uji

In [1370]:
bestInd = bestgen[0][1]
gensim = []
pj = len(bestInd)//15
lf = deepcopy(bestInd)
for j in range(pj):
    sim = []
    for k in range(15):
        sim.append(lf[0])
        del lf[0]
    gensim.append(sim)

### Read Data Uji

In [1371]:
dataUji = pd.read_excel('DatasetAI.xlsx', sheet_name=1)
dataUjiList = dataUji.values.tolist()
dataUjiBinary = convertRule(dataUjiList)

for i in range(len(dataUjiBinary)):
    del dataUjiBinary[i][14]

### Pengecekan Data Uji
Pengecakan dilakukan dengan membandingkan data uji dengan individu pohon terbaik yang didapat dari hasil generate sebelumnya

In [1372]:
listk = cekData(dataUjiBinary,gensim)

In [1374]:
for i in range(len(dataUjiBinary)):
    dataUjiBinary[i].append(listk[i])
hasil = decodeBinary(dataUjiBinary)

In [1375]:
print('Kelas :',listk)
print(*hasil, sep = "\n")

Kelas : [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
['Tinggi', 'Siang', 'Berawan', 'Rendah', 'Ya']
['Rendah', 'Siang', 'Hujan', 'Tinggi', 'Tidak']
['Normal', 'Pagi', 'Hujan', 'Tinggi', 'Tidak']
['Tinggi', 'Siang', 'Hujan', 'Normal', 'Ya']
['Rendah', 'Malam', 'Hujan', 'Rendah', 'Ya']
['Normal', 'Sore', 'Cerah', 'Normal', 'Tidak']
['Tinggi', 'Siang', 'Rintik', 'Rendah', 'Ya']
['Rendah', 'Siang', 'Cerah', 'Rendah', 'Tidak']
['Normal', 'Siang', 'Cerah', 'Rendah', 'Ya']
['Tinggi', 'Siang', 'Hujan', 'Rendah', 'Tidak']
['Tinggi', 'Sore', 'Cerah', 'Tinggi', 'Tidak']
['Normal', 'Siang', 'Cerah', 'Tinggi', 'Tidak']
['Rendah', 'Pagi', 'Berawan', 'Normal', 'Ya']
['Rendah', 'Malam', 'Rintik', 'Tinggi', 'Tidak']
['Normal', 'Siang', 'Hujan', 'Normal', 'Tidak']
['Tinggi', 'Malam', 'Cerah', 'Tinggi', 'Tidak']
['Rendah', 'Malam', 'Berawan', 'Rendah', 'Ya']
['Rendah', 'Sore', 'Rintik', 'Normal', 'Tidak']
['Tinggi', 'Siang', 'Cerah', 'Rendah', 'Tidak']
['Rendah', 'Malam', 'Rintik', 'Norma

### Simpan Hasil ke csv

In [ ]:
with open('hasilOutput.csv', "w") as outfile:
    wr = csv.writer(outfile, dialect='excel')
    wr.writerows(hasil)